---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    #criar dictionary
    lines = pd.read_csv('university_towns.txt',sep='\n', header=None)
    key = str()
    StateRegionDict = dict()
    for index, row in lines.iterrows():
        if '[edit]' in str(row):
            value = pd.Series.to_string(row, index = False).split('[edit]')[0]
        else:
            key = pd.Series.to_string(row, index = False).split(' (')[0]
            StateRegionDict[key] = value
    
    
    resp = pd.DataFrame.from_dict(data=StateRegionDict, orient='index')
    resp.reset_index(inplace=True)
    resp.columns = ['RegionName', 'State']
    resp = resp[['State', 'RegionName']]
    
    return resp

get_list_of_university_towns()

,State,RegionName
0,Wisconsin,Stevens Point
1,Indiana,South Bend
2,Vermont,Johnson
3,Kansas,Manhattan
4,Colorado,Golden
5,Texas,Lubbock
6,Rhode Island,Kingston
7,Pennsylvania,Indiana
8,North Carolina,Elon
9,Florida,Coral Gables


In [5]:
gdpDF = pd.read_excel('gdplev.xls', parse_cols='E,G', skiprows=7, index_col=0, names=['GDP in billions of chained 2009 dollars'])

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    # A recession is defined as starting with two consecutive quarters of GDP decline, 
    # and ending with two consecutive quarters of GDP growth
    # use the chained value in 2009 dollars
    recession = False
    started = []
    for i in range(1, len(gdpDF) - 1):
        if((gdpDF.iloc[i-1, 0] > gdpDF.iloc[i, 0] > gdpDF.iloc[i+1, 0]) & (not recession)):
            started.append(gdpDF.index[i])
            recession = True
        else: 
            if (gdpDF.iloc[i-1, 0] < gdpDF.iloc[i, 0] < gdpDF.iloc[i+1, 0]):
                recession = False
                
    return started

get_recession_start()



['1947q2',
 '1949q1',
 '1953q3',
 '1957q4',
 '1969q4',
 '1974q3',
 '1980q2',
 '1981q4',
 '1990q4',
 '2008q3']

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    # A recession is defined as starting with two consecutive quarters of GDP decline, 
    # and ending with two consecutive quarters of GDP growth
    # a partir de cada data acima: verificar data de fim: 2 quarters com crescimento
    recession = False
    ended = []
    for i in range(1, len(gdpDF) - 1):
        if((gdpDF.iloc[i-1, 0] > gdpDF.iloc[i, 0] > gdpDF.iloc[i+1, 0]) & (not recession)):
            recession = True
        else: 
            if (gdpDF.iloc[i-1, 0] < gdpDF.iloc[i, 0] < gdpDF.iloc[i+1, 0]):
                if(recession):
                    ended.append(gdpDF.index[i])
                recession = False      
                
    return ended

get_recession_end()

['1947q4',
 '1950q1',
 '1954q2',
 '1958q2',
 '1970q2',
 '1975q2',
 '1980q4',
 '1982q4',
 '1991q2',
 '2009q3']

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    started = get_recession_start()
    ended = get_recession_end()
    bottoms = []
    bottom = 0
    for i in range(0, len(started)):
        bottom = (gdpDF.loc[started[i] : ended[i]]).idxmin()
        bottom = bottom.get_value(0)
        bottoms.append(bottom)
        
    return bottoms

get_recession_bottom()

['1947q3',
 '1949q2',
 '1954q1',
 '1958q1',
 '1970q1',
 '1975q1',
 '1980q3',
 '1982q1',
 '1991q1',
 '2009q2']

In [19]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    ''' 
    # quarters = Q1: 1->3 Q2: 4->6 Q3: 7->9 Q4: 10->12
    homes = pd.read_csv('City_Zhvi_AllHomes.csv')


    
    
    return "ANSWER"



('IL', 'Chicago')

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"